# ハイパーパラメータの探索

> <b>ハイパーパラメータとは</b> <br>
> 機械学習では２種類のパラメータがある。学習によって更新されるものと、試行毎にユーザが渡すものである。後者はハイパーパラメータ(Hyperparameter)と呼ばれる。

機械学習の学習では、このハイパーパラメータの最適解を少ない試行回数で見つけだすことが重要。

<u>どのくらい大変そうか？</u>

(Deep Learningであると層数、学習係数、バッチサイズ,などパラメータいっぱい) x (一回の学習にかかる時間) = 現実的ではない...
しかも、リソース(サーバ, GPU等)には限りがある。

<u>どうするのか？</u>

Netfixの事例([Distributed Neural Networks with GPUs in the AWS Cloud](http://techblog.netflix.com/2014/02/distributed-neural-networks-with-gpus.html)を参考にすると、モデル並列な学習環境を用意できれば良さそう。

|手段|実現方法|
|:--|:--|
|学習をモデル並列で行って時間短縮を図る|e.g.<br> Apache Sparkを用いた分散学習|
|ハイパーパラメータを工夫して選ぶ|1. グリッドサーチ <br> 2. ランダムサーチ <br> 3. ベイズ最適化|

### ベイズ最適化

パラメータ探索は、パラメータがどんな形に沿っているかはわからないという前提があるから行われる。<br>
ここでいくつか数学記号を導入する(「それ」とか、「あれ」といった指定がしやすいため)。

* パラメータ$x$に対してパラメータが従う関数を$f(x)$とする
    * 前述した最適解とは$f(x)$が最大になる$x$は何かを推定する問題である
* ここで既に数点を評価しているとするとして、その分布を$D_{1:t} = \{x_{1:t}, f(x_{1:t})\}$とする

ベイズの定理より事後分布$P(f|D_{1:t})$は、条件付き確率$P(D_{1:t}|f)$と事前分布$P(f)$から以下のように求められる。



<div style="text-align: center;">
　$P(f|D_{1:t}) \propto P(D_{1:t}|f)P(f)$
</div>

これはつまり事前分布を仮定すると、既に評価したパラメータとからパラメータの分布を得ることができる。また事前分布$P(f)$がガウス分布に従うと仮定する。<br>
下図いくつか評価したパラメータ(黒点)と、未評価点はその信頼区間を描画している。

<img src="./img/bo1.jpg" width="500">

この得られた事後分布$f$の良さを計る指標を「獲得関数」と呼ぶ。この関数が最大となる$x$が次に調べるべきパラメータとなる。

|獲得関数                       |説明                                                                  |
|:------------------------------|:---------------------------------------------------------------------|
|Probability of Improvement (PI)|これまで得られた中でもっとも良いものを上回る確率が高いものを選ぶ      |
|Expected Improvement (EI)      |これまで得られた中でもっとも良いものを上回る幅の期待値が高いものを選ぶ|
|Upper Confidence Bound (UCB)   |信頼区間の上限が高いものを選ぶ                                        |

下図を見てみると上から下に掛けて、ベイズ最適化の過程を表す(参考文献1 Figure1より)。$t=2$では2点評価したパラメータがある。そこから次に評価すべき3点目として獲得関数が最大となる点を選んでいる($t=3$のnew observation)。以下同様に獲得関数の大きな点を調べていくことで、闇雲に探すよりは効率的に探せる。

<img src="./img/bo2.jpg" width="500">

### 実現手段

いくつか実装したものが公開されている。

|名称                                                                 |ライセンス                                               |
|:--------------------------------------------------------------------|:--------------------------------------------------------|
|[Spearmint](https://github.com/HIPS/Spearmint)                       |Academic and Non-Commercial Research Use Software License|
|[scikit-optimize](https://github.com/scikit-optimize/scikit-optimize)|BSD License                                              |
|[auto-sklearn](https://github.com/automl/auto-sklearn)               |BSD-3-Clause License                                     |

### 参考

#### ベイズ最適化

1. [A Tutorial on Bayesian Optimization of Expensive Cost Functions, with Application to Active User Modeling and Hierarchical Reinforcement Learning](https://arxiv.org/pdf/1012.2599.pdf)
2. [ベイズ的最適化の入門と応用 機械学習による機械学習の実験計画](https://www.youtube.com/watch?v=pQHWew4YYao)
3. [機械学習のためのベイズ最適化](https://book.mynavi.jp/manatee/detail/id=59393)
4. https://speakerdeck.com/tmls/keras-by-keisuke-kamataki-tmls-number-2

#### Deep Learningへの適用

1. [Bayesian optimisation and deep learning](http://www.mlss2014.com/files/defreitas_slides1.pdf)
2. [Bayesian Optimization with TensorFlow/Keras by Keisuke Kamataki - TMLS #2](https://speakerdeck.com/tmls/keras-by-keisuke-kamataki-tmls-number-2)
3. [Techniques and Systems for Training Large Neural Networks Quickly](https://stanford.edu/~rezab/nips2014workshop/slides/jeff.pdf)